In [2]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate, ChatPromptTemplate #1 prompt from string, 2 prompt from chat
from langchain.callbacks import StreamingStdOutCallbackHandler

chat = ChatOpenAI(
    model_name = "gpt-3.5-turbo",
    temperature=0.1,
    streaming=True,
    callbacks=[StreamingStdOutCallbackHandler()]
    )

poet_prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a legendary poet. You even write a poem about programming language. Your poem is about feature of language."),
    ("human", "Please write a poem about {language}.")
])

poet_chain = poet_prompt|chat



In [3]:
explain_prompt = ChatPromptTemplate.from_messages([
    ("system", "You are very excellent poetry critic.Your critique evaluates not only the content but also the structure and rhythm of the poem."),
    ("human", "would you explain this {poem}?")
])
explain_chain = explain_prompt|chat
final_chain = {"poem":poet_chain}|explain_chain

final_chain.invoke({"language":"python"})

In the realm of code, a serpent slithers free,
Python, a language of elegance and glee.
With syntax so clean, like a river's flow,
It dances through loops with a graceful glow.

Indentation its law, no curly braces in sight,
Readability reigns, like stars in the night.
Dynamic and strong, its types are but few,
Yet versatile enough to tackle all you pursue.

From web apps to AI, Python's domain,
A language of power, where creativity reigns.
Libraries abound, like treasures untold,
Pandas, NumPy, and more to behold.

So raise a toast to Python, a language divine,
Where simplicity and beauty intertwine.
In the world of code, it stands tall and true,
A legendary language, forever anew.This poem beautifully captures the essence of Python programming language with its elegant and rhythmic structure. The content praises Python for its simplicity, elegance, and versatility in the realm of coding. The use of metaphors such as comparing Python to a serpent slithering free and its syntax to a ri

AIMessageChunk(content="This poem beautifully captures the essence of Python programming language with its elegant and rhythmic structure. The content praises Python for its simplicity, elegance, and versatility in the realm of coding. The use of metaphors such as comparing Python to a serpent slithering free and its syntax to a river's flow adds depth and creativity to the poem.\n\nThe structure of the poem is well-crafted with a consistent rhyme scheme and meter, creating a smooth and flowing rhythm that mirrors the fluidity of Python's syntax. The use of enjambment in some lines adds a sense of continuity and movement, enhancing the overall flow of the poem.\n\nOverall, this poem effectively conveys the beauty and power of Python as a programming language through its content, structure, and rhythm. It celebrates Python as a legendary language that combines simplicity and beauty in a way that is both inspiring and captivating.")